<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DrugCentral-notebook:-using-psycopg2" data-toc-modified-id="DrugCentral-notebook:-using-psycopg2-1">DrugCentral notebook: using psycopg2</a></span><ul class="toc-item"><li><span><a href="#Access-database" data-toc-modified-id="Access-database-1.1">Access database</a></span></li><li><span><a href="#Get-version" data-toc-modified-id="Get-version-1.2">Get version</a></span></li><li><span><a href="#List-of-tables-(SQL)" data-toc-modified-id="List-of-tables-(SQL)-1.3">List of tables (SQL)</a></span></li><li><span><a href="#act_table_full-(SQL-EDA)" data-toc-modified-id="act_table_full-(SQL-EDA)-1.4">act_table_full (SQL EDA)</a></span></li><li><span><a href="#omop_relationship-(SQL/pandas-EDA)" data-toc-modified-id="omop_relationship-(SQL/pandas-EDA)-1.5">omop_relationship (SQL/pandas EDA)</a></span></li><li><span><a href="#Other-tables-(SQL/pandas-EDA)" data-toc-modified-id="Other-tables-(SQL/pandas-EDA)-1.6">Other tables (SQL/pandas EDA)</a></span><ul class="toc-item"><li><span><a href="#action_type" data-toc-modified-id="action_type-1.6.1">action_type</a></span></li><li><span><a href="#data_source" data-toc-modified-id="data_source-1.6.2">data_source</a></span></li><li><span><a href="#vetomop" data-toc-modified-id="vetomop-1.6.3">vetomop</a></span></li></ul></li></ul></li></ul></div>

# DrugCentral notebook: using psycopg2

In [1]:
## for notebook only 

## allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## for printing: pprint sorts dict keys by default
from pprint import pprint,pp

<div class="alert alert-block alert-danger">

Initially, I used just `psycopg2` to get familiar with the Postgres database and SQL querying. It's older, lots of documentation, and used by previous ingests (MyChem, Automat). I thought I could later load the query output into pandas ([example 1](https://naysan.ca/2020/05/31/postgresql-to-pandas/), [more detail](https://medium.com/@alestamm/importing-data-from-a-postgresql-database-to-a-pandas-dataframe-5f4bffcd8bb2)). 

But then I saw that the [dev recommends](https://github.com/psycopg/psycopg2) using psycopg(3) instead. And I thought it was easier and nicer to use [pandas's own SQL methods](https://pandas.pydata.org/pandas-docs/stable/reference/io.html#sql) to get the data into pandas. And it has `SQLAlchemy` as a dependency. 
    
So I rewrote my code to use those other packages instead. And I'm saving my old code here. 

In [2]:
## PUT required imports here

import psycopg2
import psycopg2.extras

import re
# import pandas as pd
# ## in standard python
# from urllib.parse import urlparse   ## for source_url columns

# ## NOT for parser: for viewing df only
# pd.options.display.max_columns = None
# pd.set_option('display.max_colwidth', 60)

<div class="alert alert-block alert-danger">    
    
This notebook was originally written using data accessed Dec 2025, which was dbversion number 54 and date 2023-11-01.

## Access database

Using DrugCentral's public Postgres database - details in https://drugcentral.org/download. 

I'm not sure whether to hide password. It is fully public...

References:
* [MyChem](https://github.com/biothings/mychem.info/blob/master/src/hub/dataload/sources/drugcentral/drugcentral_dump.py) parser
* [example 1](https://naysan.ca/2020/05/31/postgresql-to-pandas/)
* [more detail](https://medium.com/@alestamm/importing-data-from-a-postgresql-database-to-a-pandas-dataframe-5f4bffcd8bb2)

In [3]:
## public instance connection info
## from DrugCentral downloads page

USER = "drugman"
PASSWORD = "dosage"
HOST = "unmtid-dbs.net"
PORT = 5433
DBNAME = "drugcentral"

In [4]:
print("Connecting to the PostgreSQL database...")

conn = psycopg2.connect(
    dbname = DBNAME,
    user = USER,
    password = PASSWORD,
    host = HOST,
    port = PORT,
)

print("Connection successful")

Connecting to the PostgreSQL database...
Connection successful


In [5]:
conn

<connection object at 0x10afb6b90; dsn: 'user=drugman password=xxx dbname=drugcentral host=unmtid-dbs.net port=5433', closed: 0>

Experimented with diff cursor types, decided on `NamedTupleCursor`: can see column names (as `._fields`) and access by column name or index

Other cursor options:
- default returns tuples, no column names
- DictCursor: returns lists (unexpected!), no column names. [Stack Overflow](https://stackoverflow.com/questions/6739355/dictcursor-doesnt-seem-to-work-under-psycopg2)
- RealDictCursor: returns "RealDictRow" (dict subclass). Doesn't allow "normal" access by index ([psycopg2.extra docs](https://www.psycopg.org/docs/extras.html))

In [6]:
cursor = conn.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor)

## Get version

According to MyChem parser, the `dbversion` table contains version number and date. 

**Using date to record resource version** because it's less confusing (version number differs between live database and dump file's header)

In [7]:
## EDA

cursor.execute("SELECT * FROM dbversion")

## only 1 line in table, can use fetchone to retrieve just this line
cursor.fetchall()

[Record(version=54, dtime=datetime.datetime(2023, 11, 1, 12, 10, 57, 835000))]

In [8]:
cursor.execute("SELECT * FROM dbversion")

version_n, version_date = cursor.fetchone()
version_date = version_date.strftime("%Y-%m-%d")

version_n
version_date

54

'2023-11-01'

## List of tables (SQL)

References:
* Raw SQL query: https://stackoverflow.com/a/75752699
* Background:
  * https://www.postgresql.org/docs/current/infoschema-tables.html
  * https://www.geeksforgeeks.org/dbms/difference-between-view-and-table/

In [9]:
## grabbing table_type too for curiosity

cursor.execute("""
    SELECT table_name,table_type
    FROM information_schema.tables
    WHERE (table_schema = 'public')
""")

response_table_names = cursor.fetchall()

In [10]:
len(response_table_names)

response_table_names[0]

260

Record(table_name='SOL_PHARMACOVIGILANCE_qzJrh0Bu_FDA_products_input', table_type='BASE TABLE')

**Many more tables than expected!!** And odd names, not what I expected

**Notes on response behavior**

Acts similar to a [NamedTuple](https://www.geeksforgeeks.org/python/namedtuple-in-python/): can work with **keys** and indexes

In [11]:
## look at tables that aren't "BASE TABLE" (normal type):

for i in response_table_names:
    if i.table_type != "BASE TABLE":
        print(i)

Record(table_name='faers_top', table_type='VIEW')
Record(table_name='ob_exclusivity_view', table_type='VIEW')
Record(table_name='ob_patent_view', table_type='VIEW')
Record(table_name='omop_relationship_doid_view', table_type='VIEW')
Record(table_name='my_first_dbt_model', table_type='VIEW')
Record(table_name='my_second_dbt_model', table_type='VIEW')


`omop_relationship_doid_view` sounds interesting, so I don't want to restrict to only "BASE TABLE" in my original query. 

Now to review the full list of tables...

In [12]:
table_names = [i.table_name for i in response_table_names]
table_names = sorted(table_names)

In [13]:
## going through table names
table_names[250:275]

# for i in table_names:
#     if "test" in i:
#         print(i)

['test_2_1',
 'test_2_2',
 'test_3',
 'test_coffee_recipe',
 'test_snap',
 'vetomop',
 'vetprod',
 'vetprod2struct',
 'vetprod_type',
 'vettype']

**Observations and thoughts:**
- some sound like Gen AI/ LLM / ML stuff. Substrings:
  - `AGENT` / `AGENTIC`
  - `DEMO`
  - `GENAI`
  - `PREDICTIONLLM`
  - `model` / `MODEL`
  - `PRED` - prediction?
  - `machine_learning`
- some don't sound official/like normal use. Substrings:
  - `test`
  - `snapshot`
  - `recipe`, including `test_coffee_recipe`
- makes me wonder if users have write-access, or what happened

**Characteristics of "legit" tables**

Based on the tables previously used in parsers and other reliable-looking names, I think "legit" table names tend to only have lowercase letters, underscores, and rarely the number 2 (for "to").

Code references:
* "string shouldn't contain substrings from this list": https://stackoverflow.com/questions/58641898/check-if-string-does-not-contain-strings-from-the-list

In [14]:
name_pat = re.compile("[a-z2_]+")

possibly_good_tables = [i for i in table_names if name_pat.fullmatch(i)]

## then filter out keywords of some names that don't seem legit
odd_name_strings = ["test", "model", "snapshot"]
possibly_good_tables = [i for i in possibly_good_tables if not any(x in i for x in odd_name_strings)]

In [15]:
len(possibly_good_tables)

71

In [16]:
## reviewing these closer

possibly_good_tables[60:]

['target_dictionary',
 'target_go',
 'target_keyword',
 'td2tc',
 'tdgo2tc',
 'tdkey2tc',
 'vetomop',
 'vetprod',
 'vetprod2struct',
 'vetprod_type',
 'vettype']

## act_table_full (SQL EDA)

Link for how to get list of column names: https://www.geeksforgeeks.org/python/how-to-get-column-names-from-sqlalchemy/

In [17]:
## total number of rows
cursor.execute("""
    SELECT COUNT(*) 
    FROM act_table_full
""")
cursor.fetchone()

Record(count=20978)

In [18]:
## number of rows with action_type filled out (not None)
cursor.execute("""
    SELECT COUNT(action_type) 
    FROM act_table_full
""")
cursor.fetchone()

cursor.execute("""
    SELECT COUNT(*) 
    FROM act_table_full
    WHERE action_type IS NOT NULL
""")
cursor.fetchone()

Record(count=4360)

Record(count=4360)

In [157]:
# ## looking at number of rows filled out for other columns

# cursor.execute("""
#     SELECT COUNT(act_value) 
#     FROM act_table_full
# """)
# cursor.fetchone()

Record(count=19375)

In [26]:
## grab a row from table, see columns and example values

cursor.execute("""
    SELECT * 
    FROM act_table_full 
    WHERE act_ref_id IS NOT NULL
    LIMIT 1
""")

row = cursor.fetchall()

row[0]._asdict()

{'act_id': 215525,
 'struct_id': 296,
 'target_id': 596,
 'target_name': 'DNA topoisomerase 1',
 'target_class': 'Enzyme',
 'accession': 'P11387',
 'gene': 'TOP1',
 'swissprot': 'TOP1_HUMAN',
 'act_value': 6.561,
 'act_unit': None,
 'act_type': 'IC50',
 'act_comment': None,
 'act_source': 'SCIENTIFIC LITERATURE',
 'relation': '=',
 'moa': 1,
 'moa_source': 'SCIENTIFIC LITERATURE',
 'act_source_url': 'https://pubmed.ncbi.nlm.nih.gov/9875499',
 'moa_source_url': 'https://pubmed.ncbi.nlm.nih.gov/9875499',
 'action_type': 'INHIBITOR',
 'first_in_class': None,
 'tdl': 'Tclin',
 'act_ref_id': 612,
 'moa_ref_id': 612,
 'organism': 'Homo sapiens'}

In [30]:
cursor.execute("""
    SELECT DISTINCT organism
    FROM act_table_full
""")
row = cursor.fetchall()

len(row)

276

In [31]:
## action_type values!

cursor.execute("""
    SELECT DISTINCT action_type
    FROM act_table_full
""")

row = cursor.fetchall()

[i[0] for i in row]

['PHARMACOLOGICAL CHAPERONE',
 'INVERSE AGONIST',
 'ALLOSTERIC MODULATOR',
 'RELEASING AGENT',
 'ANTAGONIST',
 'PARTIAL AGONIST',
 'NEGATIVE ALLOSTERIC MODULATOR',
 'AGONIST',
 None,
 'POSITIVE MODULATOR',
 'GATING INHIBITOR',
 'BLOCKER',
 'NEGATIVE MODULATOR',
 'ACTIVATOR',
 'OTHER',
 'BINDING AGENT',
 'ANTIBODY BINDING',
 'ANTISENSE INHIBITOR',
 'ALLOSTERIC ANTAGONIST',
 'INHIBITOR',
 'OPENER',
 'POSITIVE ALLOSTERIC MODULATOR',
 'SUBSTRATE',
 'MODULATOR']

In [32]:
cursor.close()
conn.close()